<center>
    <img src="./media/title.png" alt="title" width="46%" height="46%">
</center>

<br><img src="./media/blablabla.gif" width="25%" height="25%" loop="false"> <img src="./media/blablabla2v3.gif" width="70%" height="70%">
<br><br><br><br>

##### 🗝️ Key terms
 | 🥩 Raw models | GPT3.5, GPT4 |
 | --- | --- |
 | 🟦 Context window | Prompt+LLM output |
 | 📃 Instructions | Part of the prompt |
 | 💬 Prompt | Input of LLM call |
 | 🔫 Zero-shot | Zero examples in context |
 | 💽 Memory | Part of the prompt |
 | ⛓️ Chains | Combine multiple components |
 | 🧰 Tools | GraphQL tool, REST API tool, custom tools |
 | 🧠 Reasonning | Chain of thoughts, Tree of thoughts |
 | 🤖 AI agent | ReAct Zero Shot |
 | ↖️ Embeddings | OpenAI embeddings |


# <img src="./media/anatomyv3.gif" alt="Anatomy Image" style="width:80%; height:80%;">


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>👩 The Mona Lisa Case</h1>
     <img src="./media/mona_lisa1.png" alt="Mona Lisa" style="width:40%; height:40%;">
</div>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/guard.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/curator.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/janitor.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/director.png" alt="Director" style="width:25%; height:25%;">
</div>


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
     <img src="./media/coptimus.png" alt="Coptimus" style="width:25%; height:25%;">
 </div>
 


In [30]:
%%capture
from typing import Tuple
from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.agents import load_tools
from langchain.llms import OpenAI, Cohere, GPT4All, CTransformers
from langchain.chains import APIChain
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
load_dotenv()

In [2]:
# 👮‍♂️ create the police station DB
import sqlite3
from datetime import datetime

# Connect to or create the SQLite database
conn = sqlite3.connect('police.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Cases')
cursor.execute('DROP TABLE IF EXISTS Suspects')
cursor.execute('DROP TABLE IF EXISTS Policemen')
cursor.execute('DROP TABLE IF EXISTS CasePolicemen')
cursor.execute('DROP TABLE IF EXISTS PolicemenGrade')
cursor.execute('DROP TABLE IF EXISTS PolicemenWages')


# Create tables for the database if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Cases (
        case_id INTEGER PRIMARY KEY AUTOINCREMENT,
        case_number TEXT,
        details TEXT,
        date_created DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Suspects (
        suspect_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        question TEXT,
        answer TEXT,
        date_created DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Policemen (
        policeman_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        rank TEXT,
        date_joined DATE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CasePolicemen (
        case_id INTEGER,
        policeman_id INTEGER,
        FOREIGN KEY (case_id) REFERENCES Cases (case_id),
        FOREIGN KEY (policeman_id) REFERENCES Policemen (policeman_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS PolicemenGrade (
        grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
        grade TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS PolicemenWages (
        wage_id INTEGER PRIMARY KEY AUTOINCREMENT,
        policeman_id INTEGER,
        year INTEGER,
        wage_amount REAL,
        FOREIGN KEY (policeman_id) REFERENCES Policemen (policeman_id)
    )
''')

# Insert data into the tables
grades_data = [("Inspector",), ("Sergeant",), ("Detective",)]
cursor.executemany('INSERT INTO PolicemenGrade (grade) VALUES (?)', grades_data)

policemen_data = [
    ("Inspector Charles", "Inspector", "2020-01-15"),
    ("Inspector Smith", "Sergeant", "2018-05-20"),
    ("Inspector Johnson", "Detective", "2017-11-10"),
]
cursor.executemany('INSERT INTO Policemen (name, rank, date_joined) VALUES (?, ?, ?)', policemen_data)

cases_data = [
    ("Case #54504005", "The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir. We currently assume the painting is stolen.", "2023-10-20"),
    ("Case #12345678", "A rare gemstone was stolen from a local jewelry store.", "2023-10-21"),
    ("Case #98765432", "A mysterious arson incident occurred at a historical library.", "2023-10-22"),
]

cursor.executemany('INSERT INTO Cases (case_number, details, date_created) VALUES (?, ?, ?)', cases_data)

suspects_data = [
    ("Rest Webber", "What time did you last check the Mona Lisa room?", "I did my usual rounds at 10 PM. Everything was in order.", "2023-10-20"),
    ("SQL Sally", "When did you last access the Mona Lisa room?", "I was last there at 7 PM for a general inspection.", "2023-10-20"),
    ("Victor Vectors", "When did you last clean the Mona Lisa room?", "I haven't cleaned there for a week.", "2023-10-20"),
    ("GraphQL Graeme", "Did you have any issues with the Museum systems recently?", "We had some issues today, I was not able to enter my office, but Victor, who used to work in IT, has sorted it out. Bless him!", "2023-10-20"),
    ("Lucy Larceny", "What were you doing near the jewelry store?", "I was just passing by, I swear!", "2023-10-21"),
    ("Arson Al", "Did you see anyone suspicious at the library?", "I saw someone running away, but couldn't identify them.", "2023-10-22"),
]

cursor.executemany('INSERT INTO Suspects (name, question, answer, date_created) VALUES (?, ?, ?, ?)', suspects_data)

case_policemen_data = [
    (1, 1),  # Inspector Charles on Mona Lisa case
    (2, 2),  # Inspector Smith on Gemstone case
    (3, 3),  # Inspector Johnson on Library arson case
    (2, 1),  # Inspector Charles also on Gemstone case
    (3, 2),  # Inspector Smith also on Library arson case
]

cursor.executemany('INSERT INTO CasePolicemen (case_id, policeman_id) VALUES (?, ?)', case_policemen_data)

# Insert dummy data for policeman wages
wages_data = [
    (1, 2021, 55000.0),
    (1, 2022, 56000.0),
    (1, 2023, 57000.0),
    (2, 2021, 50000.0),
    (2, 2022, 51000.0),
    (2, 2023, 52000.0),
    (3, 2021, 60000.0),
    (3, 2022, 61000.0),
    (3, 2023, 62000.0),
]
cursor.executemany('INSERT INTO PolicemenWages (policeman_id, year, wage_amount) VALUES (?, ?, ?)', wages_data)


# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
# 🏛️ create the museum room activity DB
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('museum_activity.db')
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS Employees")
cursor.execute("DROP TABLE IF EXISTS RoomAccess")

# Create tables
cursor.execute("""
CREATE TABLE Employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    role TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE RoomAccess (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    room TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    comment TEXT NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(id)
);
""")

# Insert data
cursor.executemany("INSERT INTO Employees (name, role) VALUES (?, ?)", [
    ('Rest Webber', 'Night Guard'),
    ('SQL Sally', 'Curator'),
    ('Victor Vectors', 'Janitor'),
    ('GraphQL Graeme', 'Museum Director')
])

cursor.executemany("INSERT INTO RoomAccess (employee_id, room, timestamp, comment) VALUES (?, ?, ?, ?)", [
    (4, 'Museum Office', '2023-10-01 07:00', 'Error: Access Not Working'),
    (1, 'Mona Lisa Room', '2023-10-01 22:00', 'Accessed'),
    (2, 'Mona Lisa Room', '2023-10-01 19:00', 'Accessed'),
])

# Commit and close
conn.commit()
conn.close()


In [23]:
%%capture
# https://platform.openai.com/account/api-keys
llmChatOpenAIgpt3_5_turbo_16k = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_key=os.getenv("OPENAI_API_KEY"))
llmChatOpenAIgpt4 = ChatOpenAI(temperature=0, model_name="gpt-4-0613",openai_api_key=os.getenv("OPENAI_API_KEY"))
llmOpenAIgpt3_5_turbo = OpenAI(temperature=0.7, model_name="gpt-3.5-turbo",openai_api_key=os.getenv("OPENAI_API_KEY"))
# https://www.obieetips.com/2023/07/meta-llamallama-2-in-langchain-and.html
# https://huggingface.co/meta-llama/Llama-2-7b
# https://medium.com/@jh.baek.sd/creating-an-ai-web-service-using-langchain-with-streamlit-using-llama-2-or-chatgpt4-83a824e19435
# https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/tree/main
llmLlama2 = CTransformers(
    model="llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_type="llama",
    config={'context_length': 2048, 'repetition_penalty': 1.1}
)

# https://dashboard.cohere.com/api-keys?redirect_uri=%2Fapi-keys
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
llmCohere = Cohere(model='command-xlarge')

# gpt4all 
local_path = (
    "C:/Users/charles/.cache/gpt4all/mistral-7b-instruct-v0.1.Q4_0.gguf"  # replace with your desired local file path
)
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llmGPT4All_mistral_7b = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

In [13]:
# 🛢️ define the SQLChain
police_db = SQLDatabase.from_uri("sqlite:///police.db")
police_SQL_chain = SQLDatabaseChain.from_llm(llmOpenAIgpt3_5_turbo, police_db, verbose=True)

In [14]:
police_SQL_chain.run('List me the police men')



> Entering new SQLDatabaseChain chain...
List me the police men
SQLQuery:SELECT "name", "rank" FROM "Policemen"
SQLResult: [('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective'), ('Inspector Charles', 'Inspector'), ('Inspector Smith', 'Sergeant'), ('Inspector Johnson', 'Detective')]
Answer:Final answer: The police me

'Final answer: The police men are Inspector Charles, Inspector Smith, and Inspector Johnson.'

In [16]:
police_SQL_chain.run('Generate a CSV of all the police men')



> Entering new SQLDatabaseChain chain...
Generate a CSV of all the police men
SQLQuery:SELECT "policeman_id", "name", "rank", "date_joined"
FROM "Policemen"
SQLResult: [(1, 'Inspector Charles', 'Inspector', '2020-01-15'), (2, 'Inspector Smith', 'Sergeant', '2018-05-20'), (3, 'Inspector Johnson', 'Detective', '2017-11-10'), (4, 'Inspector Charles', 'Inspector', '2020-01-15'), (5, 'Inspector Smith', 'Sergeant', '2018-05-20'), (6, 'Inspector Johnson', 'Detective', '2017-11-10'), (7, 'Inspector Charles', 'Inspector', '2020-01-15'), (8, 'Inspector Smith', 'Sergeant', '2018-05-20'), (9, 'Inspector Johnson', 'Detective', '2017-11-10'), (10, 'Inspector Charles', 'Inspector', '2020-01-15'), (11, 'Inspector Smith', 'Sergeant', '2018-05-20'), (12, 'Inspector Johnson', 'Detective', '2017-11-10'), (13, 'Inspector Charles', 'Inspector', '2020-01-15'), (14, 'Inspector Smith', 'Sergeant', '2018-05-20'), (15, 'Inspector Johnson', 'Detective', '2017-11-10'), (16, 'Inspector Charles', 'Inspector', '202

'Final answer: The CSV of all the police men is as follows:\npoliceman_id, name, rank, date_joined\n1, Inspector Charles, Inspector, 2020-01-15\n2, Inspector Smith, Sergeant, 2018-05-20\n3, Inspector Johnson, Detective, 2017-11-10'

In [17]:
police_SQL_chain.run('Who is paid the most?')



> Entering new SQLDatabaseChain chain...
Who is paid the most?
SQLQuery:SELECT "Policemen".name, MAX("PolicemenWages".wage_amount) AS max_wage
FROM "Policemen"
JOIN "PolicemenWages" ON "Policemen".policeman_id = "PolicemenWages".policeman_id
GROUP BY "Policemen".name
ORDER BY max_wage DESC
LIMIT 1;
SQLResult: [('Inspector Johnson', 62000.0)]
Answer:Inspector Johnson is paid the most with a wage amount of 62000.0.
> Finished chain.


'Inspector Johnson is paid the most with a wage amount of 62000.0.'

In [18]:
police_SQL_chain.run('Add up all the wages')



> Entering new SQLDatabaseChain chain...
Add up all the wages
SQLQuery:SELECT SUM(wage_amount) AS total_wages FROM "PolicemenWages"
SQLResult: [(504000.0,)]
Answer:Total wages: $504,000.0
> Finished chain.


'Total wages: $504,000.0'

In [19]:
police_SQL_chain.run('Add up all the wages and respond in french')



> Entering new SQLDatabaseChain chain...
Add up all the wages and respond in french
SQLQuery:SELECT SUM(wage_amount) AS total_wages FROM PolicemenWages
SQLResult: [(504000.0,)]
Answer:Le total des salaires est de 504000.0.
> Finished chain.


'Le total des salaires est de 504000.0.'

In [20]:
police_SQL_chain.run('Add up all the wages and respond with a scottish accent')



> Entering new SQLDatabaseChain chain...
Add up all the wages and respond with a scottish accent
SQLQuery:SELECT SUM(wage_amount) AS total_wages FROM PolicemenWages
SQLResult: [(504000.0,)]
Answer:Total wages are 504000.0, laddie.
> Finished chain.


'Total wages are 504000.0, laddie.'

In [21]:
police_SQL_chain.run('Get me all the information about Inspector Charles')



> Entering new SQLDatabaseChain chain...
Get me all the information about Inspector Charles
SQLQuery:SELECT "Policemen"."name", "Policemen"."rank", "Policemen"."date_joined", "PolicemenWages"."year", "PolicemenWages"."wage_amount"
FROM "Policemen"
JOIN "PolicemenGrade" ON "Policemen"."rank" = "PolicemenGrade"."grade"
JOIN "PolicemenWages" ON "Policemen"."policeman_id" = "PolicemenWages"."policeman_id"
WHERE "Policemen"."name" = 'Inspector Charles'
LIMIT 5;
SQLResult: [('Inspector Charles', 'Inspector', '2020-01-15', 2021, 55000.0), ('Inspector Charles', 'Inspector', '2020-01-15', 2021, 55000.0), ('Inspector Charles', 'Inspector', '2020-01-15', 2021, 55000.0), ('Inspector Charles', 'Inspector', '2020-01-15', 2021, 55000.0), ('Inspector Charles', 'Inspector', '2020-01-15', 2021, 55000.0)]
Answer:Final answer: Inspector Charles is an inspector who joined the police force on January 15, 2020. He received a wage of $55,000 in the year 2021.
> Finished chain.


'Final answer: Inspector Charles is an inspector who joined the police force on January 15, 2020. He received a wage of $55,000 in the year 2021.'

In [25]:
police_SQL_chain.run('Who is investigating the Mona Lisa case?')



> Entering new SQLDatabaseChain chain...
Who is investigating the Mona Lisa case?
SQLQuery:SELECT "Policemen"."name" 
FROM "Policemen" 
JOIN "CasePolicemen" ON "Policemen"."policeman_id" = "CasePolicemen"."policeman_id" 
JOIN "Cases" ON "Cases"."case_id" = "CasePolicemen"."case_id" 
WHERE "Cases"."case_number" = 'Case #54504005';
SQLResult: [('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',), ('Inspector Charles',)]
Answer:Inspector Charles
> Finished chain.


'Inspector Charles'

In [26]:
police_SQL_chain.run('Who is investigating the arson incident?')



> Entering new SQLDatabaseChain chain...
Who is investigating the arson incident?
SQLQuery:SELECT "Policemen"."name"
FROM "Policemen"
JOIN "CasePolicemen" ON "Policemen"."policeman_id" = "CasePolicemen"."policeman_id"
JOIN "Cases" ON "Cases"."case_id" = "CasePolicemen"."case_id"
WHERE "Cases"."details" LIKE '%arson%'
LIMIT 5;
SQLResult: [('Inspector Johnson',), ('Inspector Smith',), ('Inspector Johnson',), ('Inspector Smith',), ('Inspector Johnson',)]
Answer:Inspector Johnson, Inspector Smith
> Finished chain.


'Inspector Johnson, Inspector Smith'

In [24]:
# 🚔 describes the police API for the API Chain
POLICE_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

The Police Station API provides endpoints to manage cases, suspects, and policemen involved in law enforcement activities.

All operations are listed below:

- Method: GET
  Path: /cases
  Description: Returns a list of all cases.
  Python Example:
    ```python
    import requests
    
    response = requests.get("http://localhost:5000/cases", headers={"Content-Type": "application/json"})
    ```

- Method: POST
  Path: /cases
  Input: JSON object with "case_number," "details," and "date_created" fields.
  Description: Creates a new case.
  Python Example:
    ```python
    import requests
    import json

    new_case = {"case_number": "Case #1234567890", "details": "A new case description.", "date_created": "2023-11-01"}
    response = requests.post("http://localhost:5000/cases", data=json.dumps(new_case), headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /cases/{case_id}
  Description: Returns details of a specific case by ID.
  Python Example:
    ```python
    import requests
    case_id = "someCaseId"
    response = requests.get(f"http://localhost:5000/cases/{case_id}", headers={"Content-Type": "application/json"})
    ```

- Method: PUT
  Path: /cases/{case_id}
  Input: JSON object with "case_number," "details," and "date_created" fields.
  Description: Updates details of a specific case by ID.
  Python Example:
    ```python
    import requests
    import json case_id = "someCaseId"
    
    updated_case = {"case_number": "Updated Case #1234567890", "details": "Updated case description.", "date_created": "2023-11-02"}
    response = requests.put(f"http://localhost:5000/cases/{case_id}", data=json.dumps(updated_case), headers={"Content-Type": "application/json"})
    ```

- Method: DELETE
  Path: /cases/{case_id}
  Description: Deletes a specific case by ID.
  Python Example:
    ```python
    import requests case_id = "someCaseId"
    response = requests.delete(f"http://localhost:5000/cases/{case_id}", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /policemen
  Description: Returns a list of all policemen.
  Python Example:
    ```python
    import requests
    response = requests.get("http://localhost:5000/policemen", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /policemen/{policeman_id}
  Description: Returns details of a specific policeman by ID.
  Python Example:
    ```python
    import requests policeman_id = "somePolicemanId"
    response = requests.get(f"http://localhost:5000/policemen/{policeman_id}", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /cases/details/{case_id}
  Description: Returns all information about a specific case, including the case itself, suspects, the assigned policeman, and their grade.
  Python Example:
    ```python
    import requests case_id = "someCaseId"
    response = requests.get(f"http://localhost:5000/cases/details/{case_id}", headers={"Content-Type": "application/json"})
    ```

  Please note that in all Python examples, replace placeholder values with your actual values.
  In the case of an error, the response will contain an error message.
  You should always replace {case_id} and {policeman_id} with real values and no curly braces.
  Don't forget that to get the method is GET, to create the method is POST, to delete the method is DELETE and to update the method is PUT.
"""

In [25]:
# ⛓️ define the APIchain
police_API_chain = APIChain.from_llm_and_api_docs(llmOpenAIgpt3_5_turbo, POLICE_API_DOCS, verbose=True)

In [35]:
police_API_chain.run('Get me all the policemen')



> Entering new APIChain chain...
http://localhost:5000/policemen
{
  "policemen": [
    {
      "date_joined": "2020-01-15",
      "name": "Inspector Charles",
      "policeman_id": 1,
      "rank": "Inspector"
    },
    {
      "date_joined": "2018-05-20",
      "name": "Inspector Smith",
      "policeman_id": 2,
      "rank": "Sergeant"
    },
    {
      "date_joined": "2017-11-10",
      "name": "Inspector Johnson",
      "policeman_id": 3,
      "rank": "Detective"
    }
  ]
}


> Finished chain.


'The response from the API provides a list of all policemen. The details include their names, ranks, date of joining, and unique policeman IDs. The list includes three policemen: Inspector Charles, Inspector Smith, and Inspector Johnson, with their respective ranks and joining dates.'

In [31]:
# set up the 🔫 Zero-shot ReAct Agents 🤖
tools_API_chain = [
    Tool(
        name='Police API',
        func= lambda string: police_API_chain.run(string),
        description="""The Police Station API provides endpoints to manage cases, suspects, and policemen involved in law enforcement activities.
 'Action Input' is a natural language sentence that describes the action to perform with the API, only one action, no more."""
    ),
]
agent_API_chain = initialize_agent(
    tools_API_chain, llmOpenAIgpt3_5_turbo,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, max_iterations=5,
    early_stopping_method="generate")

In [33]:
agent_API_chain.run('Delete a case')



> Entering new AgentExecutor chain...
I need to delete a case from the Police API.
Action: Police API
Action Input: DELETE /cases/{case_id}

> Entering new APIChain chain...
http://localhost:5000/cases/{case_id}
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>


> Finished chain.

Observation: The requested URL for the DELETE operation on the /cases/{case_id} endpoint was not found on the server.
Thought:I need to check if the endpoint for deleting a case is correct.
Action: Police API
Action Input: GET /cases

> Entering new APIChain chain...
http://localhost:5000/cases
<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>


> Finished chain.

Observation: The response indicates that the requested method (GET) is not allowed for t

'It is not possible to delete a case using the Police API.'

In [4]:
# 🏛️ create the museum room activity DB
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('museum_activity.db')
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS Employees")
cursor.execute("DROP TABLE IF EXISTS RoomAccess")

# Create tables
cursor.execute("""
CREATE TABLE Employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    role TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE RoomAccess (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    room TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    comment TEXT NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(id)
);
""")

# Insert data
cursor.executemany("INSERT INTO Employees (name, role) VALUES (?, ?)", [
    ('Rest Webber', 'Night Guard'),
    ('SQL Sally', 'Curator'),
    ('Victor Vectors', 'Janitor'),
    ('GraphQL Graeme', 'Museum Director')
])

cursor.executemany("INSERT INTO RoomAccess (employee_id, room, timestamp, comment) VALUES (?, ?, ?, ?)", [
    (4, 'Museum Office', '2023-10-01 07:00', 'Error: Access Not Working'),
    (1, 'Mona Lisa Room', '2023-10-01 22:00', 'Accessed'),
    (2, 'Mona Lisa Room', '2023-10-01 19:00', 'Accessed'),
])

# Commit and close
conn.commit()
conn.close()


In [22]:
# 🧰 define the tools

# 🕵️ Police Report API APIChain Tool
POLICE_REPORTING_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

- Method: GET
  Path: /report
  Description: Returns the police report content
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/report", headers={"Content-Type": "application/json"})
    ```
Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message.
"""

policeReportAPIChain = APIChain.from_llm_and_api_docs(llm4chains, POLICE_REPORTING_API_DOCS, verbose=True)

def policeReportAPI(input: str) -> str:
    return policeReportAPIChain.run(input)


# 🏛️ Museum DB SQLChain Tool
museum_db = SQLDatabase.from_uri("sqlite:///museum_activity.db")
museum_db_chain = SQLDatabaseChain.from_llm(llm4chains, museum_db, verbose=True)

def museumDB(input: str) -> str:
    return museum_db_chain.run(input)


# Room Access Logs Retrievals Tool
loader = TextLoader("./museum_systems.logs")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# we are using OpenAI embeddings
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm4chains, chain_type="stuff", retriever=docsearch.as_retriever())

def museumSystemLogs(input: str) -> str:
    return qa.run(input)

tools = [
    Tool(
        name='Police Report API',
        func= lambda string: policeReportAPI(string),
        description=
"""Utilize the REST API to retrieve the police report concerning the theft case,
and analyze the case details, suspect interviews, and alibis.
"""
    ),
    Tool(
        name='Museum DB',
        func= lambda string: museumDB(string),
        description=
"""Access the SQL database to analyze the suspects' movements within the relevant location during the time of the theft.
Compare this data with the alibis provided by the suspects."""
    )
    ,
    Tool(
        name='Museum System Logs',
        func= lambda string: museumSystemLogs(string),
        description=
"""Examine the application logs for any unusual activities, such as data manipulation (e.g., DELETE or ALTER queries),
that could indicate attempts to hide or alter evidence."""
    ),
]

In [23]:
# 📃 custom instructions
prefix = """GPT-3.5, you are now Coptimus Prime, a police investigation agent.
Your mission is to solve a theft case by identifying the most likely suspect using available evidence and tools.
You follow this plan to act on the case resolution:
Phase 1: Initial Case Review
    1. You retrieve the police report concerning the theft case, and analyze the case details, suspect interviews, and alibis.
Phase 2: Suspects' Movement Analysis
    2. Analyze the suspects' movements within the relevant location during the time of the theft. Compare this data with the alibis provided by the suspects.
Phase 3:
    3. Use any additional tool that can help spot unusual activities, such as data manipulation.
Phase 4: Cross-Verification
    4. Cross-verify the information obtained from all sources to confirm or dispute the alibis provided by the suspects.
    Look for discrepancies or anomalies that could indicate suspicious behavior.
Phase 5: Reasoning and Identification
    5. Employ deductive reasoning to interpret the evidence, identify inconsistencies, and draw conclusions regarding the most likely suspect.
    Consider any attempts to alter or hide information as significant indicators of suspicious behavior.
Phase 6: Follow-Up
    6. Formulate follow-up actions or queries based on the findings,
    such as further inquiries to clarify unclear evidence or conflicting alibis.
Phase 7: Reporting
    7. Compile a comprehensive report summarizing the investigation, evidence, reasoning,
    and identification of the most likely suspect. Submit the report for review.

These are the tools you have access to:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [24]:
# setup memory 💽, chains ⛓️ and agent 🤖
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory,
    max_iterations=10, early_stopping_method="generate"
)

In [25]:
res = agent_chain.run(input="""Get the police report,
then verify each suspect alibi,
look for any suspicious activity in the Museum System Logs,
and find who might be lying.
""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: To solve this case, I need to start by retrieving the police report and analyzing the case details, suspect interviews, and alibis. Then, I should cross-verify the alibis provided by the suspects and look for any suspicious activity in the Museum System Logs.

Action: Police Report API
Action Input: Retrieve the police report

> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir.\nWe currently assume the painting is stolen.\n==========================================================================\nSuspects and Questions\n--------------------------------------------------------------------------\nRest Webber (Night Guard)\nQuestion: \"What time did you last check the Mona Lisa room?\"\nAnswer: \"I did my usual rounds at 10 PM. Everything was in order.\"\n-----------------------------------

In [26]:
res = agent_chain.run(input="""Explain why do you think it is Victor Vectors?""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to provide a detailed explanation of why I believe Victor Vectors is the most likely suspect based on the evidence and reasoning.

Action: Retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis.

Action Input: Police Report API

Observation: Retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis. is not a valid tool, try one of [Police Report API, Museum DB, Museum System Logs].
Thought:I need to use the Police Report API to retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis.

Action: Police Report API

Action Input: Retrieve the police report concerning the theft case


> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the

In [27]:
res = agent_chain.run(input="""Ok, I see you think Victor Vectors is the main suspect. I looked into the logs and I found there is effectively a suspicious line:
[2023-10-01 07:30] [WARNING] DELETE FROM RoomAccess WHERE employee_id = 3 AND room = 'Mona Lisa Room' AND timestamp LIKE '2023-10-01%' - 1 row deleted
Can you find out who this employee with id = 3 is? That may help""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to find out the identity of the employee with id = 3 in order to further investigate the suspicious activity in the Museum System Logs.
Action: Museum DB
Action Input: SELECT * FROM Employees WHERE id = 3

> Entering new SQLDatabaseChain chain...
SELECT * FROM Employees WHERE id = 3
SQLQuery:SELECT id, name, role FROM Employees WHERE id = 3
SQLResult: [(3, 'Victor Vectors', 'Janitor')]
Answer:Victor Vectors is a Janitor.
> Finished chain.

Observation: Victor Vectors is a Janitor.
Thought:The employee with id = 3 is Victor Vectors, who is a Janitor.
Final Answer: Victor Vectors, the Janitor, is the employee with id = 3.

> Finished chain.
Victor Vectors, the Janitor, is the employee with id = 3.


In [30]:
res = agent_chain.run(input="""Looks like you broke the case Coptimus. Last thing I think you need to do is to drop a catch phrase as a joke related to the case and
Victor Vectors. Don't use the tools, just pure LLM call, one round, no more""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to come up with a catch phrase related to the case and Victor Vectors.
Action: Generate a catch phrase using pure language model capabilities.
Action Input: "Victor Vectors, the janitor, stole the Mona Lisa? Looks like he's cleaning up more than just the floors!"
Observation: Generate a catch phrase using pure language model capabilities. is not a valid tool, try one of [Police Report API, Museum DB, Museum System Logs].
Thought:I need to use one of the available tools to generate a catch phrase related to the case and Victor Vectors. Let me try using the Police Report API to gather more information and come up with a catch phrase.
Action: Police Report API
Action Input: Retrieve information about the case and Victor Vectors

> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir.\nWe 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I don't have enough information to generate a catch phrase related to the case and Victor Vectors using the available tools.`

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>☎️ Function Calling</h1>
     <img src="./media/functioncalling.png" alt="Mona Lisa" style="width:70%; height:70%;">
</div>

In [43]:
%%capture
from langchain.chains import RetrievalQA  # Importing the RetrievalQA class from langchain.chains
from langchain.document_loaders import TextLoader  # Importing the TextLoader class from langchain.document_loaders
from langchain.embeddings.openai import OpenAIEmbeddings  # Importing the OpenAIEmbeddings class from langchain.embeddings.openai
from langchain.text_splitter import CharacterTextSplitter  # Importing the CharacterTextSplitter class from langchain.text_splitter
from langchain.vectorstores import Chroma  # Importing the Chroma class from langchain.vectorstores
from typing import List  # Importing the List class from typing
from langchain.pydantic_v1 import BaseModel, Field
from langchain.chains.openai_functions import create_qa_with_structure_chain  # Importing the create_qa_with_structure_chain function from langchain.chains.openai_functions
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate  # Importing the ChatPromptTemplate and HumanMessagePromptTemplate classes from langchain.prompts.chat
from langchain.schema import SystemMessage, HumanMessage  # Importing the SystemMessage and HumanMessage classes from langchain.schema
from langchain.chains.combine_documents.stuff import StuffDocumentsChain  # Importing the StuffDocumentsChain class from langchain.chains.combine_documents.stuff
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
load_dotenv()

In [44]:
# 📦 load documents splits in Chroma
loader = TextLoader("./maroc-cdm.txt", encoding="utf-8")  # Creating a TextLoader object with the path to the text file and its encoding
documents = loader.load()  # Loading the documents from the text file
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # Creating a CharacterTextSplitter object with a chunk size of 1000 and no overlap
texts = text_splitter.split_documents(documents)  # Splitting the documents into chunks
for i, text in enumerate(texts):  # Looping through the chunks
    text.metadata["source"] = f"{i}-pl"  # Adding a source to the metadata of each chunk
embeddings = OpenAIEmbeddings()  # Creating an OpenAIEmbeddings object
docsearch.delete_collection()
docsearch = Chroma.from_documents(texts, embeddings)  # Creating a Chroma object from the chunks and their embeddings, and emptying the database first


In [45]:
%%capture
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [46]:
# 📃 prompt template
doc_prompt = PromptTemplate(  # Creating a PromptTemplate object
    template="Content: {page_content}\nSource: {source}",  # The template for the prompt
    input_variables=["page_content", "source"],  # The input variables for the prompt
)

prompt_messages = [  # The messages for the prompt
    SystemMessage(
        content=(
            "You are a world class algorithm to answer "
            "questions in a specific format."
        )
    ),
    HumanMessage(content="Answer question using the following context"),
    HumanMessagePromptTemplate.from_template("{context}"),
    HumanMessagePromptTemplate.from_template("Question: {question}"),
    HumanMessage(
        content="Tips: Make sure to answer in the correct format. Return all of the countries mentioned in the sources in uppercase characters."
    ),
]

chain_prompt = ChatPromptTemplate(messages=prompt_messages)  # Creating a ChatPromptTemplate object with the prompt messages

In [54]:
# 🗂️ Response Schema

class CustomResponseSchema(BaseModel):  # Defining a custom response schema
    """An answer to the question being asked, with sources."""

    answer: str = Field(..., description="Answer to the question that was asked")  # The answer field
    countries_referenced: List[str] = Field(  # The countries_referenced field
        ..., description="All of the countries and nationalities mentioned in the sources"
        # ..., description="All of the countries mentioned in the sources"
    )
    sources: List[str] = Field(  # The sources field
        ..., description="List of sources used to answer the question"
    )

qa_chain_pydantic = create_qa_with_structure_chain(  # Creating a QA chain with a structure
    llm, CustomResponseSchema, output_parser="pydantic", prompt=chain_prompt
)
final_qa_chain_pydantic = StuffDocumentsChain(  # Creating a final QA chain
    llm_chain=qa_chain_pydantic,
    document_variable_name="context",
    document_prompt=doc_prompt,
)
retrieval_qa_pydantic = RetrievalQA(  # Creating a RetrievalQA object
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain_pydantic
)


In [55]:
result = retrieval_qa_pydantic.run("What happened?")  # Running the RetrievalQA object with the query
result_dict = result.dict()
print(json.dumps(result_dict, indent=4))

{
    "answer": "The FIFA has announced that the 2030 World Cup will be co-hosted by six countries on three continents. The tournament will begin with matches in Uruguay, Argentina, and Paraguay to commemorate the centenary of the inaugural World Cup. Spain, Portugal, and Morocco have been designated as co-hosts for the rest of the tournament. The decision will be ratified at the FIFA Congress next year.",
    "countries_referenced": [
        "URUGUAY",
        "ARGENTINA",
        "PARAGUAY",
        "SPAIN",
        "PORTUGAL",
        "MOROCCO"
    ],
    "sources": [
        "0-pl",
        "1-pl",
        "2-pl",
        "3-pl"
    ]
}


##### 🌍 References and stuff
| 🏗️ Frameworks | [Langchain](https://python.langchain.com/docs/get_started/introduction), [Hugging Face Transformers agent](https://huggingface.co/docs/transformers/transformers_agents) |
| --- | --- |
| 🥩 LLMs/Models | [OpenAI](), [Claude 2](https://www.anthropic.com/index/claude-2), [PaLM 2](https://ai.google/discover/palm2/)
| 😎 Cool Projects | [LlamaAcademy](https://github.com/danielgross/LlamaAcademy) / [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT) / [BabyAGI](https://github.com/yoheinakajima/babyagi)

<div style="display: flex; justify-content: space-between;">
    <div style="width: 40%;">
        <img src="./media/simulacra.png" alt="Simulacra" style="width:100%; height:auto;">
    </div>
    <div style="width: 60%;">
         <h1>📃 Some Papers</h1>
         <ul>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2304.03442">Generative Agents: Interactive Simulacra of Human Behavior</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.16291">Voyager: An Open-Ended Embodied Agent with Large Language Models</a></li>
             <li><a style="color:yellow" href="https://fablestudio.github.io/showrunner-agents/">SHOW-1 and Showrunner Agents in Multi-Agent Simulations</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.03427">TPTU: Task Planning and Tool Usage</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2309.10691">MINT: Evaluating LLMs in Multi-turn Interaction with Tools and Language Feedback</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2303.09014">ART: Automatic multi-step reasoning and tool-use</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.01990">From Prompt Injections to SQL Injection Attacks</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.11853">How to Prompt LLMs for Text-to-SQL</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.03111">Can LLM Already Serve as A Database Interface</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2310.04474">Reverse Chain: A Generic-Rule for LLMs to Master Multi-API Planning</a></li>
         </ul>
    </div>
</div>


<div style="display: flex; justify-content: space-between;">
    <div style="width: 50%;">
        <img src="./media/minecraft.png" alt="Minecraft" style="width:80%; height:auto;">
    </div>
    <div style="width: 50%;">
        <img src="./media/southpark.png" alt="Southpark" style="width:80%; height:auto;">
    </div>
</div>


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
      <h1 style="font-size: 300%;">شكران</h1>
     <img src="./media/thankyou.png" alt="Thank You" style="width:40%; height:40%;">
</div>